# Populate country and GDP
Pleae run kiva data population first.

This script take unique country code from Kiva data, load the country name and GDP for years after 2006 into country and country_gdp tables.

Instead of ORM, use SQL Expression Language to run queries. Also table reflection is been used. ORM is better suited to handle individual rows, and SQL Expression is better in run analytic queries and batch opertions. 


TODO: Logic: 

1) Get list of country_code from lenders
2) Get country code, name, lon/lat, and income_level from COUNTRY_URL
3) Get GDP for each year from wbdata API
 

In [1]:
import wbdata
import numpy as np
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine, Table, MetaData
from sqlalchemy.sql import select
import pymysql
import requests
import json

pymysql.install_as_MySQLdb()
from config import MYSQL_URL
engine = create_engine(MYSQL_URL, encoding='utf-8')
conn = engine.connect()

metadata = MetaData()

COUNTRY_URL = 'http://api.worldbank.org/v2/countries/{}?format=json'

In [2]:
# Table reflection
lender = Table('lender', metadata, autoload=True, autoload_with=engine)
country = Table('country', metadata, autoload=True, autoload_with=engine)
country_gdp = Table('country_gdp', metadata, autoload=True, autoload_with=engine)

In [3]:
country_code_list = pd.read_sql(
    select([lender.c.country_code]).distinct(),
    engine)['country_code'].tolist()

In [20]:
monthly_loan_summary = Table('monthly_loan_summary', metadata, autoload=True, autoload_with=engine)

In [29]:
df = pd.read_sql(select([monthly_loan_summary.c.year,
                   monthly_loan_summary.c.month]).distinct().order_by('year, month'), engine)

C:\Users\feng443\Anaconda3\envs\PythonData\lib\site-packages\sqlalchemy\sql\compiler.py:643: SAWarning: Can't resolve label reference 'year, month'; converting to text() (this warning may be suppressed after 10 occurrences)
  util.ellipses_string(element.element))


In [43]:
df.to_dict(orient='records')

[{'month': 4, 'year': 2006},
 {'month': 5, 'year': 2006},
 {'month': 6, 'year': 2006},
 {'month': 7, 'year': 2006},
 {'month': 8, 'year': 2006},
 {'month': 9, 'year': 2006},
 {'month': 10, 'year': 2006},
 {'month': 11, 'year': 2006},
 {'month': 12, 'year': 2006},
 {'month': 1, 'year': 2007},
 {'month': 2, 'year': 2007},
 {'month': 3, 'year': 2007},
 {'month': 4, 'year': 2007},
 {'month': 5, 'year': 2007},
 {'month': 6, 'year': 2007},
 {'month': 7, 'year': 2007},
 {'month': 8, 'year': 2007},
 {'month': 9, 'year': 2007},
 {'month': 10, 'year': 2007},
 {'month': 11, 'year': 2007},
 {'month': 12, 'year': 2007},
 {'month': 1, 'year': 2008},
 {'month': 2, 'year': 2008},
 {'month': 3, 'year': 2008},
 {'month': 4, 'year': 2008},
 {'month': 5, 'year': 2008},
 {'month': 6, 'year': 2008},
 {'month': 7, 'year': 2008},
 {'month': 8, 'year': 2008},
 {'month': 9, 'year': 2008},
 {'month': 10, 'year': 2008},
 {'month': 11, 'year': 2008},
 {'month': 12, 'year': 2008},
 {'month': 1, 'year': 2009},
 {'mo

In [30]:
import json

In [32]:
json.dumps(df.to_dict(orient='records'))

TypeError: Object of type 'int64' is not JSON serializable

## Load country table

In [4]:
country_df = pd.DataFrame(columns=[
    'country_code',
    'country_name',
    'income_level',
    'longitude',
    'latitude',
    'region'])

In [5]:
for country in country_code_list[1:]:
    if country is None:
        continue
    r = requests.get(COUNTRY_URL.format(country))
    try:
        data = r.json()[1][0]
        country_df = country_df.append(dict(
            country_code=country,
            income_level=data['incomeLevel']['value'],
            country_name=data['name'],
            latitude=data['latitude'] or None,
            longitude=data['longitude'] or None,
            region=data['region']['value'] or None), 
                         ignore_index=True)
    except:
        print('Unable to find country ' + country + ', skip.')
        

Unable to find country AI, skip.
Unable to find country AN, skip.
Unable to find country AQ, skip.
Unable to find country BV, skip.
Unable to find country CK, skip.
Unable to find country CS, skip.
Unable to find country CX, skip.
Unable to find country EH, skip.
Unable to find country FK, skip.
Unable to find country GF, skip.
Unable to find country GP, skip.
Unable to find country GS, skip.
Unable to find country IO, skip.
Unable to find country MQ, skip.
Unable to find country MS, skip.
Unable to find country NF, skip.
Unable to find country NU, skip.
Unable to find country PN, skip.
Unable to find country RE, skip.
Unable to find country SH, skip.
Unable to find country SJ, skip.
Unable to find country TF, skip.
Unable to find country TK, skip.
Unable to find country UM, skip.
Unable to find country VA, skip.
Unable to find country WF, skip.
Unable to find country YT, skip.


In [11]:
country_df = country_df.replace(r'^\s*$', np.nan, regex=True).drop_duplicates()

In [12]:
conn.execute('truncate country')
country_df.to_sql('country', engine, if_exists='append', index=False)

## Load country_gdp table

In [113]:
gdp_df = pd.DataFrame(columns=['country_code', 'year', 'gdp'])

In [114]:
for country_code in country_code_list:
    if country_code is None:
        continue
    try:
        for row in wbdata.get_data("NY.GDP.PCAP.CD", country=country_code):
            gdp_df = gdp_df.append({
                'country_code': country_code,
                'year': row['date'],
                'gdp': row['value']
            }, ignore_index=True)
    except Exception as e:
        print(e)

'NoneType' object is not iterable
'NoneType' object is not iterable
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
'NoneType' object is not iterable
list index out of range
list index out of range
list index out of range
'NoneType' object is not iterable


In [115]:
gdp_df.head()

,country_code,year,gdp
0,AD,2017,None
1,AD,2016,36988.6220301891
2,AD,2015,36038.2676044596
3,AD,2014,42294.9947269717
4,AD,2013,40619.7112977795


In [116]:
gdp_df = gdp_df.dropna()

In [119]:
# Only load GDP on and after 2006 as Kiva data go back to 2006
conn.execute('truncate country_gdp')
gdp_df.query('year >= "2006"').to_sql('country_gdp', engine, if_exists='append', index=False)
